## Memory Analysis for 5 pdf

In [1]:
from tika import parser
from elasticsearch import Elasticsearch
import os
import glob
import pandas as pd
import requests

In [148]:
import json

In [2]:
os.chdir("C:/Users/visha/Desktop/PDF")
files = glob.glob("*.pdf*")

In [3]:
print('Total Files ',len(files))

Total Files  5


In [4]:
for pdfs in files:
    print(pdfs)

resume0.pdf
resume1.pdf
resume2.pdf
resume3.pdf
resume4.pdf


In [5]:
files

['resume0.pdf', 'resume1.pdf', 'resume2.pdf', 'resume3.pdf', 'resume4.pdf']

In [46]:
def parse_files(files):
    this_loc = 1
    df = pd.DataFrame(columns = ("name", "content"))
    for file in files:
        file_data = parser.from_file(file)
        df.loc[this_loc] = file,file_data['content']
        this_loc += 1
    return df

In [47]:
df = parse_files(files)

In [48]:
df.head()

,name,content
1,resume0.pdf,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
2,resume1.pdf,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
3,resume2.pdf,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
4,resume3.pdf,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
5,resume4.pdf,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...


### memory usage in MB

In [49]:
df.memory_usage(deep=True).sum()/1000000

0.028436

In [152]:
res = requests.get('http://localhost:9200')
parsed = json.loads(res.content)
print(json.dumps(parsed, indent=2, sort_keys=True))

{
  "cluster_name": "elasticsearch",
  "cluster_uuid": "1SQrs0E6Tgytd5LrHovRTw",
  "name": "VISHAVJEET-PC",
  "tagline": "You Know, for Search",
  "version": {
    "build_date": "2020-03-26T06:34:37.794943Z",
    "build_flavor": "default",
    "build_hash": "ef48eb35cf30adf4db14086e8aabd07ef6fb113f",
    "build_snapshot": false,
    "build_type": "zip",
    "lucene_version": "8.4.0",
    "minimum_index_compatibility_version": "6.0.0-beta1",
    "minimum_wire_compatibility_version": "6.8.0",
    "number": "7.6.2"
  }
}


In [51]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

In [52]:
col_names = df.columns

In [53]:
for row_number in range(df.shape[0]):
    body = dict([(name, str(df.iloc[row_number][name])) for name in col_names])
    es.index(index = 'tika_index', doc_type='textcontent', body=body)

### search 

In [140]:
def searchContent(content):
    search_results = es.search(index = 'tika_index', doc_type='textcontent',
                               body={"_source":"name",
                                     'query':{'match_phrase':{"content":content
                                             }}})
    total_matches = search_results['hits']['total']
    print("Total matches ", total_matches['value'])
    for matches in search_results['hits']['hits']:
        print("FileName - ", matches['_source']['name'])
        print("Matching Score - ", matches['_score'])

#### Search Vishavjeet

In [141]:
searchContent("vishavjeet")

Total matches  1
FileName -  resume1.pdf
Matching Score -  1.811543


#### search Lovely

In [142]:
searchContent("lovely")

Total matches  1
FileName -  resume2.pdf
Matching Score -  1.8874922


### search aman

In [143]:
searchContent("aman")

Total matches  3
FileName -  resume2.pdf
Matching Score -  1.5165889
FileName -  resume3.pdf
Matching Score -  1.08482
FileName -  resume4.pdf
Matching Score -  1.0411689


### Velore

In [144]:
searchContent("velore")

Total matches  1
FileName -  resume4.pdf
Matching Score -  1.811543


### search non existing name

In [145]:
searchContent("aabrakadabra")

Total matches  0


### search framework

In [146]:
searchContent("framework")

Total matches  1
FileName -  resume0.pdf
Matching Score -  2.0084016


### Total index Size

In [193]:
res = requests.get('http://localhost:9200/tika_index/_stats')
index_size = json.loads(res.content)
print("Total index size - ",index_size["indices"]["tika_index"]["total"]["store"])

Total index size -  {'size_in_bytes': 41403}
